## Exploring Conduit and Contractile Strains AND Strain Rates!

## Updated August 30th 2023

This notebook is for exploring the differences in conduit and contractile phase strains between patient groups

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from hrs_23_figures import *

from sklearn import metrics

regions=['global', 'roof', 'sept', 'lat', 'ant', 'post']

In [ ]:
path="/home/csi20/Dropbox/phd/Data/kiru_demographics_df_omitAFimaging.csv"

In [ ]:
cases_df = pd.read_csv(path)

In [ ]:
case_list=list(cases_df["Case"])
remove_cases=["01", "05", "09", "14", "20", "21", "25", "26", "27", "31"]
remove_cases=[f"CT-CRT/case{i}" for i in remove_cases]
append_ebr=["01", "04", "06"]
append_ebr=[f"EBR/case{i}" for i in append_ebr]
remove_cases+=append_ebr

case_list = [i for i in case_list if i not in remove_cases]

In [ ]:
## Load all strains

dataPath = "/home/csi20local/Data/RG_CT_Cases"
dataPath = "/home/csi20/Dropbox/phd/Data/RG_CT_Cases"

# N_cases, N_strain_phases
strains = np.zeros((len(case_list), 3))
SRs = np.zeros((len(case_list), 3))

for i, case in enumerate(case_list):
    filepath = get_tsffd_trackingpath(case)
    
    ## tsffd area strains
    strains[i, :] = np.loadtxt(f"{filepath}/area_global_strains_all_phases_v3.txt")
    SRs[i, :] = np.loadtxt(f"{filepath}/area_global_SRs_all_phases_v3.txt")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,6), sharey=True, sharex=True)

for i, ax in enumerate(fig.axes):
    ax.hist(np.abs(strains[:, i]))
    
fig.suptitle("Strains", fontsize=20)

ax1.set_title("Reservoir", fontsize=17)
ax2.set_title("Conduit", fontsize=17)
ax3.set_title("Contractile", fontsize=17)

ax1.set_xlim(0.0, 60)

plt.tight_layout()

We see that Contractile and Conduit strains are similar in magnitude

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,6), sharey=True, sharex=True)

for i, ax in enumerate(fig.axes):
    ax.hist(np.abs(SRs[:, i]))
    
fig.suptitle("SR", fontsize=20)

ax1.set_title("Reservoir", fontsize=17)
ax2.set_title("Conduit", fontsize=17)
ax3.set_title("Contractile", fontsize=17)

ax2.set_xlim(0.0, 35)
ax3.set_xlim(0.0, 35)

plt.tight_layout()

We see that the contractile SR is generally greater amplitude than that of reservoir function

In [ ]:
af_cases_idx = cases_df[cases_df['af_num']==1]['Case'].index

In [ ]:
naf_cases_idx = cases_df[cases_df['af_num']==0]['Case'].index

In [ ]:
naf_cases = [cases_df['Case'].iloc[i] for i in naf_cases_idx]
af_cases = [cases_df['Case'].iloc[i] for i in af_cases_idx]

In [ ]:
naf_cases_SRsubset=[]
af_cases_SRsubset=[]

for case in case_list:
    if case in naf_cases:
        naf_cases_SRsubset.append(case)
        
    elif case in af_cases:
        af_cases_SRsubset.append(case)

In [ ]:
af_cases_SRsubset

In [ ]:
# arguments for naf and af cases

naf_inds_SRsubset=[]
af_inds_SRsubset=[]

for case in naf_cases_SRsubset:
    
    ans=case_list.index(case)
    naf_inds_SRsubset.append(ans)
    
for case in af_cases_SRsubset:
    
    ans=case_list.index(case)
    af_inds_SRsubset.append(ans)

In [ ]:
## Define area strains

res_af = [strains[i, 0] for i in af_inds_SRsubset]
res_naf = [strains[i, 0] for i in naf_inds_SRsubset]

cond_af = [strains[i, 1] for i in af_inds_SRsubset]
cond_naf = [strains[i, 1] for i in naf_inds_SRsubset]

cont_af = [strains[i, 2] for i in af_inds_SRsubset]
cont_naf = [strains[i, 2] for i in naf_inds_SRsubset]

In [ ]:
## Define area SRs

SR_res_af = [SRs[i, 0] for i in af_inds_SRsubset]
SR_res_naf = [SRs[i, 0] for i in naf_inds_SRsubset]

SR_cond_af = [SRs[i, 1] for i in af_inds_SRsubset]
SR_cond_naf = [SRs[i, 1] for i in naf_inds_SRsubset]

SR_cont_af = [SRs[i, 2] for i in af_inds_SRsubset]
SR_cont_naf = [SRs[i, 2] for i in naf_inds_SRsubset]

In [ ]:
fig, ((ax1, ax2, ax3), 
      (ax4, ax5, ax6)) = plt.subplots(2,3, figsize=(8,8), sharey=True)

sns.boxplot(data=[res_naf, res_af], ax=ax1)
sns.boxplot(data=[cond_naf, cond_af], ax=ax2)
sns.boxplot(data=[cont_naf, cont_af], ax=ax3)

sns.boxplot(data=[SR_res_naf, SR_res_af], ax=ax4)
sns.boxplot(data=[SR_cond_naf, SR_cond_af], ax=ax5)
sns.boxplot(data=[SR_cont_naf, SR_cont_af], ax=ax6)

ax1.set_ylabel("Area Strain", fontsize=16)
ax4.set_ylabel("Area Strain Rate", fontsize=16)

ax4.set_xlabel("Reservoir", fontsize=16)
ax5.set_xlabel("Conduit", fontsize=16)
ax6.set_xlabel("Contractile", fontsize=16)

fig.suptitle("TSFFD GLOBAL STRAIN", fontsize=20)

In [ ]:
## print p values

res_p = stats.ttest_ind(res_af, res_naf)
cond_p = stats.ttest_ind(cond_af, cond_naf)
cont_p = stats.ttest_ind(cont_af, cont_naf)

print("tsffd strains\n")

print("area strains")
print("Reservoir p-val:\t", res_p)
print("Conduit p-val:\t\t", cond_p)
print("Contractile p-val:\t", cont_p)

res_p = stats.ttest_ind(SR_res_af, SR_res_naf)
cond_p = stats.ttest_ind(SR_cond_af, SR_cond_naf)
cont_p = stats.ttest_ind(SR_cont_af, SR_cont_naf)

print("\nSR")
print("Reservoir SR p-val:\t", res_p)
print("Conduit SR p-val:\t\t", cond_p)
print("Contractile SR p-val:\t", cont_p)

In [ ]:
fig, ((ax1, ax2, ax3)) = plt.subplots(1,3, figsize=(6,6), sharey=True)

for i, ax in enumerate(fig.axes):
    ax.boxplot(strains[:, i])

In [ ]:
fig, ((ax1, ax2, ax3)) = plt.subplots(1,3, figsize=(6,6), sharey=True)

for i, ax in enumerate(fig.axes):
    ax.boxplot(np.abs(SRs[:, i]))